In [1]:
import time, queue, socket, sqlite3, datetime, threading  

class Tello:
    def __init__(self):
        self.socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM) 
        self.socket.bind(('', 8889))   
        self.db_queue  = queue.Queue() # cache flight data
        self.cmd_queue = queue.Queue() 
        self.cmd_lock = threading.Lock()  
        self.MAX_TIME_OUT = 15         # It must be longer than 10 sec, give time to "take off" command.   
        self.MAX_RETRY = 2
        self.state = {}
        threading.Thread(target=self.flight_logger, daemon=True).start() 
        threading.Thread(target=self.receiver     , daemon=True).start()
        threading.Thread(target=self.sender       , daemon=True).start()   
        threading.Thread(target=self.update_state , daemon=True).start()   
    
    def is_cmd_locked(self): 
        if self.cmd_lock.acquire(timeout=self.MAX_TIME_OUT):  
            self.cmd_lock.release()
            return True
        else: 
            return False
    
    def command(self, cmd):
        self.cmd_queue.put(cmd)  
     
    def flight_logger(self):
        con = sqlite3.connect(f'Tello_flight_log_{datetime.datetime.fromtimestamp(time.time()).strftime("%Y%m%d_%H%M%S")}.db')  
        cur = con.cursor() 
        cur.execute('CREATE TABLE commands(timestamp REAL, command TEXT, who TEXT);')
        cur.execute('CREATE TABLE   states(timestamp REAL, log     TEXT          );') 
        print('Flight Data Recording Begins ~') 
        while 1: 
            operation = self.db_queue.get() 
            if   operation == 'commit': 
                con.commit()
                print('Flight Data Saved ~') 
            elif operation == 'close': 
                con.close()  
                print('Flight Data Recording Ends ~') 
                break
            else: 
                cur.execute(operation) 
         
    def receiver(self):  
        while True: 
            bytes_, address = self.socket.recvfrom(1024)  
            if bytes_ == b'ok':  
                self.cmd_lock.release() 
            else:
                print('[ Station ]:', bytes_)
            try:
                self.db_queue.put('INSERT INTO commands(timestamp, command, who) VALUES({}, "{}", "{}");'.format(time.time(), bytes_.decode(), "Tello")) 
            except UnicodeDecodeError as e:
                print('Decoding Error that could be ignored~')
    
    def sender(self, debug=True): 
        tello_address = ('192.168.10.1', 8889)  
        while True: 
            cmd = self.cmd_queue.get()   
            self.cmd_lock.acquire()  
            self.db_queue.put(f'INSERT INTO commands(timestamp, command, who) VALUES({time.time()}, "{cmd}", "Station");')  
            self.socket.sendto(cmd.encode('utf-8'), tello_address)
            for i in range(self.MAX_RETRY):   
                if self.is_cmd_locked(): 
                    break
                else:
                    if debug: print(f'Failed command: "{cmd}", Failure sequence: {i+1}.')
                    self.socket.sendto(cmd.encode('utf-8'), tello_address) 
            if self.is_cmd_locked():
                print(f'Success with "{cmd}".')       
            elif debug:
                print(f'Stop retry: "{cmd}", Maximum re-tries: {self.MAX_RETRY}.')
                self.cmd_lock.release() 
              
    def update_state(self):
        UDP = socket.socket(socket.AF_INET, socket.SOCK_DGRAM) 
        UDP.bind(('', 8890))  
        while True: 
            bytes_, address = UDP.recvfrom(1024)
            str_ = bytes_.decode() 
            self.db_queue.put('INSERT INTO states(timestamp, log) VALUES({},"{}");'.format(time.time(), str_))  
            state = str_.split(';')
            state.pop() 
            self.state.update(dict([s.split(':') for s in state]))
            
tello = Tello() 

Flight Data Recording Begins ~


In [2]:
tello.command('command')

Success with "command".


In [3]:
print("It may take 10 seconds to take off.") 
tello.command('takeoff') 

It may take 10 seconds to take off.


In [4]:
tello.command('forward 50')
tello.command('left 50'   )
tello.command('back 50'   )
tello.command('right 50'  ) 

In [5]:
tello.command('land') 

Success with "takeoff".
Success with "forward 50".
Success with "left 50".
Success with "back 50".
Success with "right 50".
Success with "land".


In [6]:
tello.db_queue.put('commit')
tello.db_queue.put('close')

Flight Data Saved ~
Flight Data Recording Ends ~
